In [1]:
# My created .py files for modularization
import env
import os
import wrangle as w
# Ignore Warning
import warnings
warnings.filterwarnings("ignore")
# Array and Dataframes
import numpy as np
import pandas as pd
# Imputer
from sklearn.impute import SimpleImputer
# Evaluation: Visualization
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
# Evaluation: Statistical Analysis
from scipy import stats
# Modeling: Scaling
from sklearn.preprocessing import QuantileTransformer
# Modeling
from sklearn.model_selection import GridSearchCV
# Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
# delimit columns showed for ease
pd.set_option('display.max_columns', None)

In [3]:
# defining dataframe from wrangle
zillow = w.zillow()

File exists pulling from system.


## Data Previews & Insight

In [4]:
# sample
zillow.sample(5)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid
765124,4.0,3.0,2099.0,729000.0,1991.0,7693.14,6111.0,261.0
62302,5.0,3.0,2269.0,823258.0,1977.0,9304.16,6059.0,261.0
1113591,4.0,2.0,2663.0,109925.0,1925.0,1679.07,6037.0,261.0
1921261,3.0,2.0,1212.0,135869.0,1972.0,1944.47,6037.0,261.0
437659,3.0,2.0,1570.0,277476.0,1959.0,3300.28,6037.0,261.0


In [5]:
# explore columns: nunique, unique, isna, dtype
w.summarize(zillow)

,Column Name,Number of Unique Values,Unique Values,Number of Null Values,dtype
7,propertylandusetypeid,1,[261.0],0,float64
6,fips,3,"[6037.0, 6059.0, 6111.0]",0,float64
0,bedroomcnt,19,"[0.0, 4.0, 3.0, 5.0, 2.0, 1.0, 6.0, 7.0, 8.0, ...",11,float64
1,bathroomcnt,38,"[0.0, 2.0, 4.0, 1.0, 2.5, 3.5, 3.0, 5.5, 4.5, ...",11,float64
4,yearbuilt,153,"[nan, 2005.0, 2011.0, 1926.0, 1972.0, 1973.0, ...",9337,float64
2,calculatedfinishedsquarefeet,10580,"[nan, 3633.0, 1620.0, 2077.0, 1200.0, 171.0, 2...",8484,float64
3,taxvaluedollarcnt,592269,"[27516.0, 10.0, 2108.0, 296425.0, 124.0, 84777...",493,float64
5,taxamount,918838,"[nan, 174.21, 6941.39, 10244.94, 7924.68, 8034...",4442,float64


In [6]:
# info
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2152863 entries, 0 to 2152862
Data columns (total 8 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
 7   propertylandusetypeid         float64
dtypes: float64(8)
memory usage: 147.8 MB


In [7]:
# describe
zillow.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid
count,2.152852e+06,2.152852e+06,2.144379e+06,2.152370e+06,2.143526e+06,2.148421e+06,2.152863e+06,2152863.0
mean,3.287196e+00,2.230688e+00,1.862855e+03,4.618962e+05,1.960950e+03,5.634866e+03,6.048377e+03,261.0
std,9.547544e-01,9.992796e-01,1.222125e+03,6.996760e+05,2.216220e+01,8.178910e+03,2.043329e+01,0.0
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.801000e+03,1.850000e+00,6.037000e+03,261.0
25%,3.000000e+00,2.000000e+00,1.257000e+03,1.881702e+05,1.949000e+03,2.534980e+03,6.037000e+03,261.0
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.276710e+05,1.958000e+03,4.108950e+03,6.037000e+03,261.0
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.345270e+05,1.976000e+03,6.414320e+03,6.059000e+03,261.0
max,2.500000e+01,3.200000e+01,9.525760e+05,9.842891e+07,2.016000e+03,1.337756e+06,6.111000e+03,261.0


In [8]:
# checking amount of outliers
zillow[zillow.yearbuilt.between(1800,1900)]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid
207,3.0,1.0,1319.0,114727.0,1890.0,1270.80,6111.0,261.0
3523,2.0,1.0,782.0,115971.0,1895.0,NaN,6037.0,261.0
3549,2.0,1.0,840.0,393000.0,1885.0,4850.66,6037.0,261.0
3551,3.0,1.0,1674.0,418000.0,1887.0,5197.58,6037.0,261.0
3917,4.0,2.0,1488.0,226336.0,1900.0,2757.69,6037.0,261.0
...,...,...,...,...,...,...,...,...
2147922,2.0,2.0,1276.0,85584.0,1894.0,1459.52,6037.0,261.0
2148015,2.0,1.0,1270.0,284667.0,1895.0,3486.72,6037.0,261.0
2148041,3.0,2.0,1134.0,28737.0,1900.0,378.50,6037.0,261.0
2148840,3.0,1.0,1469.0,147329.0,1885.0,1974.96,6037.0,261.0


In [9]:
yearmean = zillow['yearbuilt'].mean().round(0)
zillow['yearbuilt'].fillna(yearmean,inplace=True)

In [10]:
# drop all values with missing bedroomcnt values
zillow[zillow.bedroomcnt.isnull()].dropna(inplace=True)

In [11]:
zillow.taxamount.max()

1337755.86